In [1]:
import os

import numpy as np
import pytorch_lightning as pl
import torch
import torch_geometric.data as geom_data
import torch_geometric.transforms as T
from lightning.pytorch import loggers as pl_loggers
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torch.utils.tensorboard import SummaryWriter
from torch_geometric.data import Data
from torch_geometric.datasets import Planetoid

%reload_ext autoreload
%autoreload 2

#### Configure device

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if device != "cpu":
    torch.set_float32_matmul_precision("high")

# device = "cpu"
# torch.set_float32_matmul_precision("high")

num_workers = os.cpu_count()
device, num_workers

(device(type='cuda', index=0), 12)

### Directory configuration and load data

In [3]:
from utils.dataset.CUHKSZ_AcademicGraph import CUHKSZ_AcademicGraph

cwd = os.getcwd()
tb_logging_dir = os.path.join(cwd, "lightning_logs")
exp_name = "AG-GraphSAGE-inductive"
exp_dir = os.path.join(tb_logging_dir, exp_name)
dataset_dir = os.path.join(cwd, "dataset", "CUHKSZ_AcademicGraph")
if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)

AGDataset = CUHKSZ_AcademicGraph(dataset_dir, with_title=True, with_label=True)

AGDataset[0]

D:\GitHub\aml-project\GNN\zyq\dataset\CUHKSZ_AcademicGraph\raw\CUHKSZ_AcademicGraph_Rawdata.zip
D:\GitHub\aml-project\GNN\zyq\dataset\CUHKSZ_AcademicGraph\raw\CUHKSZ_AcademicGraph-rawdata_released


Data(x=[6614, 768], edge_index=[2, 12330], y=[6614], title=[6614], train_mask=[6614], val_mask=[6614], test_mask=[6614])

#### Note that if numebr of epoch is large, the kernel will stuck after training! Need to load the saved model mannually!

In [4]:
from utils.model.RS_GraphSAGE import GraphSage

dataset = AGDataset

early_stop_callback = EarlyStopping(
    monitor="val_loss", min_delta=0.00, patience=10, verbose=False, mode="min"
)
hparams = {
    "aggregator_type": "mean",
    "HIDDEN_DIM": [112, 16],  # size of the embedding
    "BATCH_SIZE": 64,
    "LEARNING_RATE": 0.001,
    "NUM_NEIGHBORS": [10, 10],
}  # The number of neighbors in each order of sampling

tb_logger = pl_loggers.TensorBoardLogger(tb_logging_dir, name=exp_name)

trainer = pl.Trainer(
    max_epochs=100,
    callbacks=[early_stop_callback],
    logger=tb_logger,
    log_every_n_steps=1,
    # accelerator="cpu"
    # num_sanity_val_steps = 0
)

version_dir = os.path.join(exp_dir, "version_" + str(trainer.logger.version))
writer_acc = SummaryWriter(log_dir=version_dir)
writer_loss = SummaryWriter(log_dir=version_dir)

checkpoint_dir = os.path.join(version_dir, "checkpoints")
print("Saving checkpoints to", checkpoint_dir)

SAGEmodel = GraphSage(
    dataset=dataset,
    input_dim=dataset.num_features,
    hparams=hparams,
    writer_acc=writer_acc,
    writer_loss=writer_loss,
).to(device)
SAGEmodel.data_processing_inductive()

trainer.fit(SAGEmodel)
trainer.test(SAGEmodel)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Saving checkpoints to D:\GitHub\aml-project\GNN\zyq\lightning_logs\AG-GraphSAGE-inductive\version_7\checkpoints


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type              | Params
--------------------------------------------
0 | aggr  | MeanAggregation   | 0     
1 | model | Sequential_2ea012 | 175 K 
--------------------------------------------
175 K     Trainable params
0         Non-trainable params
175 K     Total params
0.703     Total estimated model params size (MB)


Data Processing Done on: cuda:0


Sanity Checking: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
D:\Anaconda\envs\torchGNN\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │     0.971908654696952     │
│         test_loss         │    0.4680634140968323     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.4680634140968323, 'test_acc': 0.971908654696952}]

#### Upload trained model

In [4]:
from utils.model.RS_GraphSAGE import GraphSage

hparams = {
    "aggregator_type": "mean",
    "HIDDEN_DIM": [112, 16],  # size of the embedding
    "BATCH_SIZE": 64,
    "LEARNING_RATE": 0.001,
    "NUM_NEIGHBORS": [10, 10],
}
dataset = AGDataset
checkpoint_dir = "..\\lightning_logs\\AG-GraphSAGE-inductive\\best\\checkpoints"
checkpoint_file = os.path.join(checkpoint_dir, os.listdir(checkpoint_dir)[0])
Loaded_model = GraphSage.load_from_checkpoint(
    checkpoint_file, dataset=dataset, input_dim=dataset.num_features, hparams=hparams
)
Loaded_model.data_processing_inductive()

Data Processing Done on: cpu


#### To see the mini-batch's number of negative edges:

In [6]:
tmpLoader = Loaded_model.train_dataloader()
sample = next(iter(tmpLoader))
sample

Data(x=[3207, 768], edge_index=[2, 6430], edge_label=[6430], train_mask=[3207], n_id=[3207], e_id=[6430], input_id=[64], batch_size=64)

#### To see the number of training positive edges:

In [7]:
Loaded_model.pos_data["train"]

Data(x=[6614, 768], edge_index=[2, 4218], edge_label=[4218], train_mask=[6614])

#### Recommendation for a given node

In [5]:
import torch.nn.functional as F


def recommendation_by_inductive_model(A, model):
    # Return recommedation for A in the given graph
    neg_graph = model.neg_data["all"]
    candidate = torch.zeros(neg_graph.num_edges)

    candidate = neg_graph.edge_index[1, (neg_graph.edge_index[0] == A)].cpu()
    candidate.sort()
    num_candidates = len(candidate)
    u = torch.ones(num_candidates, dtype=torch.int) * A
    v = torch.Tensor(candidate)
    candidate_link = torch.stack((u, v))

    embedding = model(model.pos_data["all"].x, model.pos_data["all"].edge_index)

    scores = np.zeros(candidate.max() + 1) - 1e8

    scores[v] = F.sigmoid((embedding[u] * embedding[v]).sum(dim=1)).detach().numpy()

    rank_idx = np.argsort(scores)[::-1]

    pos_graph = model.pos_data["all"]
    print("Recommendation for node", A, "with title", pos_graph.title[A])
    for k in range(10):
        print(rank_idx[k], scores[rank_idx[k]], pos_graph.title[rank_idx[k]])


recommendation_by_inductive_model(1370, Loaded_model)

Recommendation for node 1370 with title Non-linear matrix completion
3756 0.9992935657501221 Simultaneous tensor subspace selection and clustering: the equivalence of high order svd and k-means clustering
119 0.997686505317688 R1-PCA: rotational invariant L1-norm principal component analysis for robust subspace factorization
2914 0.9970500469207764 Two-dimensional PCA: a new approach to appearance-based face representation and recognition
6150 0.9966974258422852 Exactly Robust Kernel Principal Component Analysis
1815 0.9961254000663757 K-means clustering via principal component analysis
1276 0.9959867596626282 Linearized cluster assignment via spectral ordering
2741 0.9943919777870178 Polynomial Matrix Completion for Missing Data Imputation and Transductive Learning
5431 0.9941216111183167 Matrix completion by deep matrix factorization
3514 0.9937432408332825 Graph-Laplacian PCA: Closed-Form Solution and Robustness
2205 0.993690013885498 Robust Matrix Completion via Joint Schatten p-No

In [9]:
Loaded_model.pos_data["all"].test_mask.argwhere()

tensor([[   2],
        [   7],
        [  11],
        ...,
        [6596],
        [6597],
        [6613]])

In [6]:
recommendation_by_inductive_model(1676, Loaded_model)

Recommendation for node 1676 with title A Semismooth Newton Stochastic Proximal Point Algorithm with Variance Reduction
2484 0.9964107871055603 Convergence analysis of alternating direction method of multipliers for a family of nonconvex problems
2794 0.9962867498397827 On the Convergence Rate of Dual Ascent Methods for Linearly Constrained Convex Minimization
6343 0.9961708188056946 Convergence to good non-optimal critical points in the training of neural networks: Gradient descent optimization with one random initialization overcomes all bad non-global local minima with high probability
5822 0.9957494735717773 On the Superlinear Convergence of Interior-Point Algorithms for a General Class of Problems
3284 0.9956661462783813 On Iteration Complexity of a First-Order Primal-Dual Method for Nonlinear Convex Cone Programming
2648 0.9953540563583374 On the convergence of the iteration sequence in primal-dual interior-point methods
6603 0.995296061038971 Error Bound and Convergence Analysis